# ASISTENTES DE OPENAI

https://platform.openai.com/docs/assistants/overview

https://github.com/openai/openai-python/blob/main/examples/assistant.py

La API de Asistentes te permite construir asistentes de inteligencia artificial dentro de tus propias aplicaciones. Un Asistente tiene instrucciones y puede aprovechar modelos, herramientas y conocimiento para responder a las consultas de los usuarios. Actualmente, la API de Asistentes admite tres tipos de herramientas: Intérprete de Código, Recuperación y Llamado de Funciones. En el futuro, planeamos lanzar más herramientas construidas por OpenAI y permitir que proporcionen sus propias herramientas en nuestra plataforma.

Puedes explorar las capacidades de la API de Asistentes utilizando el playground de Asistentes o construyendo una integración paso a paso descrita en esta guía. A un alto nivel, una integración típica de la API de Asistentes tiene el siguiente flujo:

- Crear un Asistente en la API definiendo sus instrucciones personalizadas y eligiendo un modelo. Si es útil, habilita herramientas como Intérprete de Código, Recuperación y Llamado de Funciones.
- Crear un Hilo cuando un usuario inicia una conversación.
- Añadir Mensajes al Hilo a medida que el usuario hace preguntas.
- Ejecutar el Asistente en el Hilo para desencadenar respuestas. Esto llama automáticamente a las herramientas relevantes.

In [1]:
import openai
import time


GPT_MODEL = "gpt-3.5-turbo-0613"
import os
from dotenv import load_dotenv

# Cargar variables de entorno desde .env
load_dotenv()

# Acceder a la API key
api_key = os.getenv("API_KEY")
openai.api_key = api_key

## Paso 1: Crear un Asistente

Un Asistente representa una entidad que puede ser configurada para responder a los Mensajes de los usuarios utilizando varios parámetros como:

- Instructions: cómo debe comportarse o responder el Asistente y el modelo.
- Model: puedes especificar cualquier modelo de GPT-3.5 o GPT-4, incluyendo modelos afinados. La herramienta de Recuperación requiere los modelos gpt-3.5-turbo-1106 y gpt-4-1106-preview.
- Tools: la API soporta Intérprete de Código y Recuperación que están construidos y alojados por OpenAI.
- Functions: la API te permite definir firmas de funciones personalizadas, con un comportamiento similar a nuestra característica de llamado a funciones.

En este ejemplo, estamos creando un Asistente que es un tutor personal de matemáticas, con la herramienta de Intérprete de Código habilitada.

In [6]:
#client = openai.OpenAI()

assistant = openai.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-1106-preview",
    #api_key= api
)

## Paso 2: Crear un Hilo ---> Thread

Un Hilo representa una conversación. Recomendamos crear un Hilo por usuario tan pronto como el usuario inicie la conversación. Pasa cualquier contexto específico del usuario y archivos en este hilo creando Mensajes.

Los Hilos no tienen un límite de tamaño. Puedes añadir tantos Mensajes como desees a un Hilo. El Asistente se asegurará de que las solicitudes al modelo se ajusten dentro de la ventana de contexto máxima, utilizando técnicas de optimización relevantes como la truncación, las cuales hemos probado extensamente con ChatGPT. Cuando usas la API de Asistentes, delegas el control sobre cuántos tokens de entrada se pasan al modelo para cualquier ejecución dada, esto significa que tienes menos control sobre el costo de ejecutar tu Asistente en algunos casos, pero no tienes que lidiar con la complejidad de gestionar la ventana de contexto tú mismo.


In [7]:
thread = openai.beta.threads.create()

## Paso 3 añadir un mensaje al hilo

Un Mensaje contiene texto y, opcionalmente, cualquier archivo que permitas que el usuario suba. Los Mensajes necesitan ser añadidos a un Hilo específico. Añadir imágenes a través de objetos de mensaje como en Completados de Chat usando GPT-4 con Visión no es soportado hoy en día, pero planeamos añadir soporte para ellos en los próximos meses. Aún puedes subir imágenes y procesarlas a través de la recuperación.

In [9]:
message = openai.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
)

In [10]:
# thread_messages = openai.beta.threads.messages.list("thread_abc123")
# print(thread_messages.data)

## Paso 4: Ejecutar el Asistente

Para que el Asistente responda al mensaje del usuario, necesitas crear una Ejecución. Esto hace que el Asistente lea el Hilo y decida si llamar a herramientas (si están habilitadas) o simplemente usar el modelo para responder mejor a la consulta. A medida que la ejecución avanza, el asistente añade Mensajes al hilo con el rol="assistant". El Asistente también decidirá automáticamente qué Mensajes anteriores incluir en la ventana de contexto para el modelo. 

Esto tiene un impacto tanto en la tarificación como en el rendimiento del modelo. El enfoque actual ha sido optimizado basado en lo que aprendimos al construir ChatGPT y probablemente evolucionará con el tiempo.

Opcionalmente, puedes pasar nuevas instrucciones al Asistente al crear la Ejecución, pero ten en cuenta que estas instrucciones anulan las instrucciones predeterminadas del Asistente.

In [11]:
run = openai.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Jane Doe. The user has a premium account.",
)

## Paso 5: Verificar el estado de la Ejecución
Por defecto, una Ejecución entra en estado de en cola. Puedes recuperar periódicamente la Ejecución para verificar su estado y ver si ha pasado a completado.

## Paso 6: Mostrar la Respuesta del Asistente
Una vez que la Ejecución se completa, puedes listar los Mensajes añadidos al Hilo por el Asistente.

In [18]:
print("checking assistant status. ")
while True:
    run = openai.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    if run.status == "completed":
        print("done!")
        messages = openai.beta.threads.messages.list(thread_id=thread.id)

        print("messages: ")
        for message in messages:
            assert message.content[0].type == "text"
            print({"role": message.role, "message": message.content[0].text.value})

        openai.beta.assistants.delete(assistant.id)

        break
    else:
        print("in progress...")
        time.sleep(20)

checking assistant status. 


in progress...
in progress...
in progress...
in progress...


KeyboardInterrupt: 

In [14]:
messages = openai.beta.threads.messages.list(
  thread_id=thread.id
)

In [16]:
messages.

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_hURyu6ICTFPF5OdnXklMfKww', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1701775679, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_5ePyNXLCjzj1Wkxp6PuFhuwr'), ThreadMessage(id='msg_44J4n9xzGflnsaAGttH8e7Ft', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1701775585, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_5ePyNXLCjzj1Wkxp6PuFhuwr')], object='list', first_id='msg_hURyu6ICTFPF5OdnXklMfKww', last_id='msg_44J4n9xzGflnsaAGttH8e7Ft', has_more=False)